In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.cluster import KMeansClusterer,cosine_distance
from sklearn.decomposition import LatentDirichletAllocation
import json
from numpy.random import shuffle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.cluster import KMeansClusterer, cosine_distance
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.cluster import KMeans

In [82]:
#K means based on Cosine distance

def cluster_kmean(data):
    data=pd.read_csv(data, header=0, names=['commentText', 'label'])

    # generating the tfidf matrix
    tfidf_vect = TfidfVectorizer(stop_words='english',\
                             min_df=5)
    #splitting the data into training and testing set.
    
    train=data[:800]
    test=data[800:999]

    dtm=tfidf_vect.fit_transform(train["commentText"])
    
    #number of cluster that we want is 2
    num_clusters=2
    clusterer = KMeansClusterer(num_clusters, cosine_distance, repeats=1)
    clusters = clusterer.cluster(dtm.toarray(),assign_clusters=True)
    
    #working on the test data
    test_dtm = tfidf_vect.transform(test["commentText"])
    
    predicted = [clusterer.classify(v) for v in test_dtm.toarray()]
  
    #creating the confusion matrix
    confusion_df = pd.DataFrame(list(zip(test['label'].values, predicted)),\
                           columns = ["label", "cluster"])
    
    #creating a cross tab
    x=pd.crosstab( index=confusion_df.cluster, columns=confusion_df.label)
   
    #selecting 2 labels
    cluster_dict = {0:0, 1:1}
    predicted_target=[cluster_dict[i] for i in predicted]
    
    #printing the metrics
    print(metrics.classification_report(test["label"], predicted_target))

In [86]:
def alt_cluster_kmean(data): #euclidean distance with sklearn, first approach was nltk and cosine distance
    
    data=pd.read_csv(data, header=0, names=['commentText', 'label'])
    #train, test = train_test_split(data, test_size=0.2, random_state=0)
    #data=data.dropna(subset = ['commentText', 'label'])

    #Splitting the data into training and testing
    train=data[:800]
    test=data[800:999]
    
    #generating the tfidf matrix 
    tfidf_vect = TfidfVectorizer(stop_words="english",\
                             min_df=5) 
    dtm= tfidf_vect.fit_transform(train["commentText"])
    
    #setting the number of clusters to 2
    num_clusters=2
    
    km = KMeans(n_clusters=num_clusters, n_init=20).fit(dtm)
    clusters = km.labels_.tolist()
    
    test_dtm = tfidf_vect.transform(test['commentText'])
    
    #predicting the labels of the test data 
    predicted = km.predict(test_dtm)
    
    #generating a confusion matrix
    confusion_df = pd.DataFrame(list(zip(test['label'], predicted)),\
                            columns = ["label", "cluster"])
    
    #creating a cross tab
    x=pd.crosstab( index=confusion_df.cluster, columns=confusion_df.label)
    cluster_dict={0:0, 1:1}
      
    predicted_target=[cluster_dict[i] for i in predicted]

    #print(test[:100])
    #printing the metrics
    print(metrics.classification_report(test["label"], predicted_target))

In [90]:
if __name__ == "__main__":
    #cluster_kmean("/Users/divyaj_podar/Desktop/Final Project/s10.csv")
    #cluster_kmean("/Users/divyaj_podar/Desktop/Final Project/iphone.csv")

    alt_cluster_kmean("/Users/divyaj_podar/Desktop/Final Project/s10.csv")
    #alt_cluster_kmean("/Users/divyaj_podar/Desktop/Final Project/iphone.csv")

    #first method working alot better for iphone

              precision    recall  f1-score   support

         0.0       0.19      0.88      0.32        40
         1.0       0.71      0.08      0.14       159

   micro avg       0.24      0.24      0.24       199
   macro avg       0.45      0.48      0.23       199
weighted avg       0.60      0.24      0.17       199

